In [90]:
import os
from dotenv import load_dotenv
load_dotenv()
project_path = os.getenv("PROJECT_PATH")

In [91]:
%run "{project_path}\llm_custom_apps\llm_apps\oh_sheet_its_spark\utils.ipynb"

13


In [92]:
xls_col_info = get_excel_col_header(f"{test_data_path}\SalesData1.xlsx")
xls_col_info

{'SalesInfo': {'StoreID': {'ColumnID': 'A'},
  'SalesYear': {'ColumnID': 'B'},
  'SalesValue': {'ColumnID': 'C'},
  'StoreLocation': {'ColumnID': 'D'},
  'Owner': {'ColumnID': 'E'}},
 'RegionInfo': {'ID': {'ColumnID': 'A'}, 'Location': {'ColumnID': 'B'}},
 'StoreInfo': {'ID': {'ColumnID': 'A'},
  'RegionID': {'ColumnID': 'B'},
  'StoreOwner': {'ColumnID': 'C'}},
 'SaleGrowthReport1': {'StoreID': {'ColumnID': 'A'},
  'AverageSales': {'ColumnID': 'B'},
  'MinSales': {'ColumnID': 'C'},
  'MaxSales': {'ColumnID': 'D'}},
 'SaleGrowthReport2': {'StoreLocation-SalesYear': {'ColumnID': 'A'},
  'Year': {'ColumnID': 'B'},
  'Location': {'ColumnID': 'C'},
  'AverageSales': {'ColumnID': 'D'},
  'MinSales': {'ColumnID': 'E'},
  'MaxSales': {'ColumnID': 'F'}}}

In [93]:
xls_str = process_excel_columns_xlwings(f"{test_data_path}\SalesData1.xlsx",xls_col_info)
xls_str

{'SalesInfo': {'StoreID': {'ColumnID': 'A', 'ColumnValue': '11'},
  'SalesYear': {'ColumnID': 'B', 'ColumnValue': '2020'},
  'SalesValue': {'ColumnID': 'C', 'ColumnValue': '15789'},
  'StoreLocation': {'ColumnID': 'D',
   'ColumnValue': '=IFERROR(INDEX(RegionInfo!B:B, MATCH(INDEX(StoreInfo!B:B, MATCH(A2, StoreInfo!A:A, 0)), RegionInfo!A:A, 0)), "")'},
  'Owner': {'ColumnID': 'E',
   'ColumnValue': '=IFERROR(INDEX(StoreInfo!C:C, MATCH(A2, StoreInfo!A:A, 0)), "")'}},
 'RegionInfo': {'ID': {'ColumnID': 'A', 'ColumnValue': '1'},
  'Location': {'ColumnID': 'B', 'ColumnValue': 'Halifax'}},
 'StoreInfo': {'ID': {'ColumnID': 'A', 'ColumnValue': '11'},
  'RegionID': {'ColumnID': 'B', 'ColumnValue': '1'},
  'StoreOwner': {'ColumnID': 'C', 'ColumnValue': 'Ethan Carter'}},
 'SaleGrowthReport1': {'StoreID': {'ColumnID': 'A',
   'ColumnValue': '=_xlfn.UNIQUE(_xlfn._xlws.FILTER(SalesInfo!A:A, SalesInfo!A:A<>""))'},
  'AverageSales': {'ColumnID': 'B',
   'ColumnValue': '=AVERAGEIF(SalesInfo!A:A, A2, S

In [94]:
messages = [
    {"role": "system", "content": sheet_spark_prompt},
    {"role": "user", "content": f"Convert the excel logic representation to equivalent pyspark code : {xls_str}"},
  ]

In [95]:
#generate_responses("JasjyotSinghJaswal/code_qwen_1.5_7b_quant",messages)

In [96]:
generate_responses_inference("Qwen/Qwen2.5-Coder-32B-Instruct",messages)

Preparing input conversation...
Generating response...
Response:
User: 
You are a helpful assistant to a Data Scientist who is working on a project to transform Excel Spreadsheets to PySpark Dataframes. 
The Data Scientist has provided you with a dictionary of dictionaries which contains details about Excel spreadsheets.
Each dictionary has only one key which is the name of the Sheet and the value is a dictionary with multiple keys.
Each key is the name of the Column Header in the Excel Spreadsheet. The value is again a dictionary with two keys.
The first key is 'ColumnID' with the value being the associated ColumnID in Excel i.e. A, B, C, or etc.
The second key is 'ColumnValue' with the value being the associated formula for generating the column or the hardcoded value in the absence of the formula.
The dictionary looks something like:
{{'SheetNum1': {'X': {'ColumnID': 'A', 'ColumnValue': 11},
   'Y': {'ColumnID': 'B', 'ColumnValue': 2020},
   'Z': {'ColumnID': 'C', 'ColumnValue': 157

In [99]:
respllama = generate_responses_inference("meta-llama/Llama-3.2-1B",messages)

Preparing input conversation...
Generating response...
Response:
User: 
You are a helpful assistant to a Data Scientist who is working on a project to transform Excel Spreadsheets to PySpark Dataframes. 
The Data Scientist has provided you with a dictionary of dictionaries which contains details about Excel spreadsheets.
Each dictionary has only one key which is the name of the Sheet and the value is a dictionary with multiple keys.
Each key is the name of the Column Header in the Excel Spreadsheet. The value is again a dictionary with two keys.
The first key is 'ColumnID' with the value being the associated ColumnID in Excel i.e. A, B, C, or etc.
The second key is 'ColumnValue' with the value being the associated formula for generating the column or the hardcoded value in the absence of the formula.
The dictionary looks something like:
{{'SheetNum1': {'X': {'ColumnID': 'A', 'ColumnValue': 11},
   'Y': {'ColumnID': 'B', 'ColumnValue': 2020},
   'Z': {'ColumnID': 'C', 'ColumnValue': 157

In [100]:
respllama